In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# remember to clone the repo
# !git clone https://github.com/MatthewZhang473/LoRA_Qwen0.5B.git

In [3]:
import os

# Change the current directory to your repository in Google Drive
os.chdir('/content/drive/MyDrive/LoRA_Qwen0.5B')

# Now you can run scripts or access files within your repository
# For example, to list the files in the current directory:
!ls

colabs		   llmtime.pdf	qwen.pdf   src		 wandb
fine_tuned_models  main.pdf	Readme.md  test-trainer


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys
from peft import LoraConfig, TaskType, get_peft_model, PeftModel
from src.data.data_loader import LotkaVolterraDataset, encode, decode
from datetime import datetime

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Set pad_token to eos_token if it's not set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


def preprocess_function(examples):
    processed_inputs = tokenizer(examples["text"],
                     truncation=True,
                     padding=True) # Add padding here
    processed_inputs["labels"] = processed_inputs["input_ids"].copy()
    return processed_inputs

# Load dataset
raw_dataset = LotkaVolterraDataset()
trajectories = raw_dataset.trajectories
texts = encode(trajectories)
dataset = Dataset.from_dict({"text": texts})

# Split dataset into training and validation sets
split_dataset = dataset.train_test_split(test_size=0.2)
tokenized_dataset = split_dataset.map(preprocess_function, batched=True, remove_columns=["text"])

train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
lora_config = LoraConfig(
    r=12,
    target_modules=["q_proj", "v_proj"],
    task_type=TaskType.CAUSAL_LM,
    lora_alpha=32,
    lora_dropout=0.05
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 811,008 || all params: 494,843,776 || trainable%: 0.1639


In [ ]:
training_args = TrainingArguments(
    "test-trainer",
    fp16=True,
    num_train_epochs=10.0,
    learning_rate=5e-5,
    eval_strategy="steps",   # ← enables evaluation during training
    eval_steps=100,                # ← evaluate every 100 steps
    logging_strategy="steps",      # ← log every X steps
    logging_steps=100,             # ← match eval frequency for W&B updates
    report_to="wandb"
)


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset, # Use the training dataset
    eval_dataset=eval_dataset, # Use the evaluation dataset
    data_collator=data_collator
)

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
trainer.train()


Step,Training Loss,Validation Loss
100,0.650400,0.653843
200,0.638900,0.646505
300,0.631700,0.638384


Step,Training Loss,Validation Loss
100,0.650400,0.653843
200,0.638900,0.646505
300,0.631700,0.638384
400,0.626300,0.636935
500,0.622000,0.630086
600,0.617600,0.627147
700,0.614600,0.625552
800,0.612300,0.623092
900,0.609700,0.619813
1000,0.608200,0.618981


TrainOutput(global_step=1000, training_loss=0.623155345916748, metrics={'train_runtime': 375.101, 'train_samples_per_second': 21.328, 'train_steps_per_second': 2.666, 'total_flos': 2.071329942528e+16, 'train_loss': 0.623155345916748, 'epoch': 10.0})

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
lora_settings = f"r{lora_config.r}_alpha{lora_config.lora_alpha}"
output_dir = f"./fine_tuned_models/{timestamp}_{lora_settings}"
os.makedirs(output_dir, exist_ok=True)
trainer.save_model(output_dir)
print(f"Model saved to {output_dir}")

Model saved to ./fine_tuned_models/20251017_113135_r12_alpha32


## Load & Inference

In [7]:
# Load the base model
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# List available checkpoints
checkpoints_dir = "./fine_tuned_models/"
if os.path.exists(checkpoints_dir):
    available_checkpoints = [d for d in os.listdir(checkpoints_dir) if os.path.isdir(os.path.join(checkpoints_dir, d))]
    print("Available checkpoints:")
    for checkpoint in available_checkpoints:
        print(f"- {checkpoint}")
else:
    print(f"No checkpoints found in {checkpoints_dir}")

# Specify the directory where you saved the LoRA adapter weights you want to load
# Replace "YYYYMMDD_HHMMSS_rX_alphaY" with the actual timestamp and settings of the model you want to load
lora_model_path = "./fine_tuned_models/20251017_113135_r12_alpha32" # <--- Change this to the checkpoint you want to load

# Load the LoRA adapter weights on top of the base model
try:
    model = PeftModel.from_pretrained(base_model, lora_model_path)
    print(f"LoRA fine-tuned model loaded successfully from {lora_model_path}!")
except Exception as e:
    print(f"Error loading model from {lora_model_path}: {e}")
    model = None # Set model to None if loading fails

Available checkpoints:
- 20251017_101834_r6_alpha32
- 20251017_113135_r12_alpha32
LoRA fine-tuned model loaded successfully from ./fine_tuned_models/20251017_113135_r12_alpha32!


In [8]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=12, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=12, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_feature